In [4]:
!pip install transformers datasets torch

In [5]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [6]:
dataset = load_dataset("squad_v2")

# Pisahkan dataset menjadi train & validation
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [7]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_data(example):
    # Format pertanyaan dan konteks untuk T5
    question = example["question"]
    context = example["context"]
    answer = example["answers"]["text"][0] if example["answers"]["text"] else ""

    input_text = f"question: {question} context: {context}"
    target_text = f"{answer}"

    encoding = tokenizer(input_text, padding="max_length", truncation=True, max_length=512)
    target_encoding = tokenizer(target_text, padding="max_length", truncation=True, max_length=64)

    encoding["labels"] = target_encoding["input_ids"]
    return encoding

# Tokenisasi dataset
train_dataset = train_dataset.map(preprocess_data, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(preprocess_data, remove_columns=eval_dataset.column_names)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [8]:
# Bagi dataset menjadi 50% untuk training pertama dan 50% untuk lanjutan
train_dataset_split = train_dataset.train_test_split(test_size=0.5, seed=42)

train_dataset_1 = train_dataset_split["train"]
train_dataset_2 = train_dataset_split["test"]

In [9]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./t5-squad2-checkpoint",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,  # Simpan hanya 2 checkpoint terbaru
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_1,  # Training dengan 50% data pertama
    eval_dataset=eval_dataset
)

trainer.train()

# Simpan model setelah 50% training selesai
trainer.save_model("./t5-squad2-checkpoint")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: garingguring6 (garingguring6-smk-negeri-12-surabaya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.034600,0.031770


Epoch,Training Loss,Validation Loss
1,0.034600,0.031770
2,0.031000,0.029942
3,0.028200,0.029877


In [11]:
# 3️⃣ Simpan Tokenizer SETELAH Training 50%
tokenizer.save_pretrained("./t5-tokenizer-50percent")

('./t5-tokenizer-50percent/tokenizer_config.json',
 './t5-tokenizer-50percent/special_tokens_map.json',
 './t5-tokenizer-50percent/spiece.model',
 './t5-tokenizer-50percent/added_tokens.json')

In [14]:
import shutil

# Buat file zip untuk checkpoint model
shutil.make_archive("/content/t5-squad2-checkpoint", 'zip', "/content/t5-squad2-checkpoint")

# Buat file zip untuk tokenizer 50%
shutil.make_archive("/content/t5-tokenizer-50percent", 'zip', "/content/t5-tokenizer-50percent")


'/content/t5-tokenizer-50percent.zip'

In [16]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
# Pastikan sudah zip dulu
!zip -r /content/t5-squad2-checkpoint.zip /content/t5-squad2-checkpoint
!zip -r /content/t5-tokenizer-50percent.zip /content/t5-tokenizer-50percent

# Buat folder di Google Drive (jika belum ada)
!mkdir -p "/content/drive/My Drive/T5_Checkpoints"

# Pindahkan file ke Google Drive
!mv /content/t5-squad2-checkpoint.zip "/content/drive/My Drive/T5_Checkpoints/"
!mv /content/t5-tokenizer-50percent.zip "/content/drive/My Drive/T5_Checkpoints/"


  adding: content/t5-squad2-checkpoint/ (stored 0%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/ (stored 0%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/scheduler.pt (deflated 55%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/training_args.bin (deflated 52%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/model.safetensors (deflated 8%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/config.json (deflated 63%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/optimizer.pt (deflated 7%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/rng_state.pth (deflated 25%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/trainer_state.json (deflated 73%)
  adding: content/t5-squad2-checkpoint/checkpoint-16290/generation_config.json (deflated 29%)
  adding: content/t5-squad2-checkpoint/runs/ (stored 0%)
  adding: content/t5-squad2-checkpoint/runs/Mar26_14-25-16_1fd5f434af07/ (stored 0%)
  adding: content/t5-squad2-checkpoint/ru

In [18]:
!ls "/content/drive/My Drive/T5_Checkpoints/"


t5-squad2-checkpoint.zip  t5-tokenizer-50percent.zip


In [19]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Path ke folder yang sudah diekstrak
model_path = "/content/t5-squad2-checkpoint"
tokenizer_path = "/content/t5-tokenizer-50percent"

# Load tokenizer dan model
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

print("Model dan tokenizer berhasil dimuat!")


Model dan tokenizer berhasil dimuat!


In [35]:
def answer_question(question, context):
    # Format input sesuai format T5
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate jawaban
    output_ids = model.generate(input_ids)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer

# Contoh tes
context = "Albert Einstein was a theoretical physicist who developed the theory of relativity."
question = "Who developed the theory of relativity?"

print("Pertanyaan:", question)
print("Jawaban:", answer_question(question, context))

Pertanyaan: Who developed the theory of relativity?
Jawaban: Albert Einstein


In [13]:
model = T5ForConditionalGeneration.from_pretrained("./t5-squad2-checkpoint")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_2,  # Training dengan sisa 50% data
    eval_dataset=eval_dataset
)

trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.save_model("./t5-squad2-final")
tokenizer.save_pretrained("./t5-squad2-final")


# **===============================================================================================**

In [29]:
!pip install duckduckgo-search transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.7 MB/s eta 0:00:00


In [1]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load tokenizer & model dari checkpoint yang sudah kita simpan
tokenizer = T5Tokenizer.from_pretrained("/content/t5-tokenizer-50percent")
model = T5ForConditionalGeneration.from_pretrained("/content/t5-squad2-checkpoint")

print("✅ Model dan tokenizer berhasil dimuat!")


✅ Model dan tokenizer berhasil dimuat!


In [22]:
from duckduckgo_search import DDGS
from deep_translator import GoogleTranslator

def search_context(query):
    """Cari informasi dari DuckDuckGo, ambil beberapa hasil, dan terjemahkan ke bahasa Inggris."""
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)  # Ambil 3 hasil

    context = " ".join([r["body"] for r in results if "body" in r])  # Gabungkan hasil
    if not context:
        return "Context not found."

    # Terjemahkan ke bahasa Inggris
    translator = GoogleTranslator(source="auto", target="en")
    translated_context = translator.translate(context)

    return translated_context

# Tes ambil konteks dan terjemahkan
question = "Who first landed on the moon?"
context = search_context(question)

print("Translated Context:", context)



Translated Context: It marked the first time that humans landed on the Moon. Commander Neil Armstrong and Lunar Module Pilot Buzz Aldrin landed the Apollo Lunar Module Eagle on July 20, 1969, at 20:17 UTC, and Armstrong became the first person to step onto the Moon's surface six hours and 39 minutes later, on July 21 at 02:56 UTC. It reads, "Here men from the planet Earth first set foot upon the moon. July 1969 A.D. We came in peace for all mankind." Armstrong and Aldrin blast off and dock with Collins in Columbia. Collins later says that "for the first time," he "really felt that we were going to carry this thing off." The crew splashes down off Hawaii on ... 1. Luna 9 [USSR, February 3, 1966, uncrewed]: The first successful moon landing. Before astronauts ever set foot on the Moon, the Soviets, and Americans both sent fleets of robotic spacecraft to scout and pave the way. Soviet Union's Luna 9 was the first spacecraft to achieve a moon landing, and, in general, survivable landing on

In [23]:
def generate_answer(question, context):
    """Jawab pertanyaan dengan model T5"""
    input_text = f"question: {question}  context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate jawaban
    output_ids = model.generate(input_ids, max_length=128)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Contoh pertanyaan dan konteks
question = "Who first landed on the moon?"
# Coba jawab pertanyaan
answer = generate_answer(question, context)
print("Jawaban:", answer)


Jawaban: humans
